In [ ]:
import matplotlib.pyplot as plt
from sunpy.map import Map
from sunpy.net import Fido, attrs as a
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib.patches as patches
import os
import drms

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
jsoc_email = "lillinadol@gmail.com"

result = Fido.search(
    a.Time("2011-02-15 00:00:00", "2011-02-15 00:01:00"),
    a.Sample(1 * u.hour),
    a.jsoc.Series("hmi.sharp_cea_720s"),
    a.jsoc.PrimeKey("HARPNUM", 377),
    a.jsoc.Notify(jsoc_email),
    a.jsoc.Segment("Br"),
)
print(result)

In [ ]:
file = Fido.fetch(result)
sharp_map = Map(file)

In [ ]:
# Extract the active region number from metadata
ar_number = sharp_map.meta.get('HARPNUM')
print(f"Active Region Number (HARPNUM): {ar_number}")

In [ ]:
sharp_map.meta.get('NOAA_AR')


In [ ]:
sharp_map.meta.get('NOAA_NUM')


In [ ]:
sharp_map.meta.get('NOAAARS')

In [ ]:
path_aia171 = (
    "/Users/lilli/sunpy/data/aia.lev1.171A_2011_02_15T00_00_00.34Z.image_lev1.fits"
)

aia_image = Map(path_aia171).rotate()

In [ ]:
left_corner = sharp_map.bottom_left_coord
right_corner = sharp_map.top_right_coord
aia_small = aia_image.submap(left_corner, top_right=right_corner)

In [ ]:
from mhsxtrapy.field2d import Field2dData, FluxBalanceState

px_arcsec = (
    np.arctan(
        (
            sharp_map.fits_header["CDELT1"]
            * np.pi
            / 180
            * sharp_map.fits_header["RSUN_REF"]
        )
        / sharp_map.fits_header["DSUN_OBS"]
    )
    * 180
    / np.pi
    * 3600
)
py_arcsec = (
    np.arctan(
        (
            sharp_map.fits_header["CDELT2"]
            * np.pi
            / 180
            * sharp_map.fits_header["RSUN_REF"]
        )
        / sharp_map.fits_header["DSUN_OBS"]
    )
    * 180
    / np.pi
    * 3600
)

px_radians = px_arcsec / ((3600 * 360) / (2 * np.pi))
py_radians = py_arcsec / ((3600 * 360) / (2 * np.pi))

dist_Mm = sharp_map.fits_header["DSUN_OBS"] * 10**-6
px = px_radians * dist_Mm
py = py_radians * dist_Mm

ny, nx = sharp_map.data.shape
bz = sharp_map.data


xmin = 0.0
ymin = 0.0

xmax = nx * px
ymax = ny * py

x = np.arange(nx) * (xmax - xmin) / (nx - 1) - xmin
y = np.arange(ny) * (ymax - ymin) / (ny - 1) - ymin

zmin = 0.0

zmax = 40.0

# pz = np.float64(90.0 * 10**-3)
pz = max(px, py)
nz = int(np.floor(zmax / pz))
z = np.arange(nz) * (zmax - zmin) / (nz - 1) - zmin

nf = int(min(nx, ny))

sharp_data = Field2dData(
    nx,
    ny,
    nz,
    nf,
    px,
    py,
    pz,
    x,
    y,
    z,
    bz,
    flux_balance_state=FluxBalanceState.BALANCED,
)

In [ ]:
from scipy.interpolate import griddata

nx = aia_small.data.shape[1]
ny = aia_small.data.shape[0]

x = np.arange(nx) * (sharp_data.x[-1] - sharp_data.x[0]) / (nx - 1) - sharp_data.x[0]
y = np.arange(ny) * (sharp_data.y[-1] - sharp_data.y[0]) / (ny - 1) - sharp_data.y[0]

xv_fine, yv_fine = np.meshgrid(sharp_data.x, sharp_data.y)
xv, yv = np.meshgrid(x, y)

print(
    yv.flatten().shape,
    xv.flatten().shape,
    yv_fine.flatten().shape,
    xv_fine.flatten().shape,
)
print(aia_small.data.flatten().shape)
print(np.column_stack((yv.flatten(), xv.flatten())).shape)


AIA_higherres = griddata(
    np.column_stack((yv.flatten(), xv.flatten())),
    aia_small.data.flatten(),
    np.column_stack((yv_fine.flatten(), xv_fine.flatten())),
    method="cubic",
).reshape(sharp_data.bz.shape)

In [ ]:
from mhsxtrapy.field3d import calculate_magfield, WhichSolution
from mhsxtrapy.field2d import maximal_a

In [ ]:
maximal_a(sharp_data, alpha=0.01, b=1.0)

In [ ]:
sharp_extra = calculate_magfield(
    sharp_data,
    alpha=0.01,
    a=0.4,
    which_solution=WhichSolution.ASYMP,
    b=1.0,
    z0=2.0,
    deltaz=0.2,
)

In [ ]:
sharp_extra.nz

In [ ]:
sharp_extra.y[-1]

In [ ]:
sharp_extra.bz.max()

In [ ]:
from mhsxtrapy.plotting.vis import (
    plot_magnetogram_3D,
    plot_ddensity_xy,
    plot_ddensity_z,
    plot_dpressure_xy,
    plot_dpressure_z,
)

In [ ]:
plot_magnetogram_3D(sharp_extra, view="side", footpoints="active-regions")

In [ ]:
z_plot = 6

In [ ]:
plot_dpressure_xy(sharp_extra, z=z_plot)

In [ ]:
plot_ddensity_xy(sharp_extra, z=z_plot)

In [ ]:
plot_dpressure_z(sharp_extra)

In [ ]:
plot_ddensity_z(sharp_extra)